In [14]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
df=pd.read_csv('Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
df.drop(['RowNumber','CustomerId','Surname'],axis=1,inplace=True)

In [8]:
label_encoder_gender=LabelEncoder()
df['Gender']=label_encoder_gender.fit_transform(df['Gender'])

onehot_encoder_geo=OneHotEncoder()
encoded_geo=onehot_encoder_geo.fit_transform(df[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(encoded_geo,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))

In [9]:
df.drop('Geography',axis=1,inplace=True)

In [10]:
df=pd.concat([df,geo_encoded_df],axis=1)

In [12]:
X=df.drop('Exited',axis=1)
y=df['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [15]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)
with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [16]:
# Define a function to create a model and try different parameters(kerasclassifier)

def create_model(neurons=32,layers=1):
    model=Sequential()
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neurons,activation='relu'))
    
    model.add(Dense(1,activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    return model

In [21]:
#Creating Keras Classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [22]:
# Define the grid search parameters
param_grid={
    'neurons':[16,32,64,128],
    'layers':[1,2,3],
    'epochs':[50,100]
}

In [23]:
grid= GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=-1,cv=3)
grid_result=grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

2025-06-17 17:13:06.145850: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-17 17:13:06.156593: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-17 17:13:06.200798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-17 17:13:06.242009: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-17 17:13:06.253397: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
E0000 00:00:1750160586.284556   17657 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-17 17:13:06.289

KeyboardInterrupt: 